In [33]:
from sqlalchemy import create_engine, Column, Integer, Float, desc
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [17]:
import pandas as pd
import os

In [18]:
database_file = os.path.join(os.path.abspath(os.getcwd()), 'data.db')
database_file

'c:\\Users\\yTonomura\\Documents\\after_2020\\13.Python\\other_basis\\sqlalchemy\\data.db'

In [19]:
engine = create_engine('sqlite:///' + database_file, convert_unicode=True, echo=True)
# engine = create_engine('sqlite:///:memory:', convert_unicode=True, echo=True)
Base = declarative_base()

In [20]:
db_session = scoped_session(
   sessionmaker(
      autocommit = False,
      autoflush = False,
      bind = engine
   )
)
# Session = sessionmaker(autocommit = False, autoflush = False, bind = engine)
# session = Session()

In [21]:
class Wine(Base):
   __tablename__ = 'wine_class'
   id = Column(Integer, primary_key=True)
   wine_class = Column(Integer, unique=False)
   alcohol = Column(Float, unique=False)
   ash = Column(Float, unique=False)
   hue = Column(Float, unique=False)
   proline = Column(Integer, unique=False)

   def __init__(self, wine_class=None, alcohol=None, ash=None, hue=None, proline=None):
      self.wine_class = wine_class
      self.alcohol = alcohol
      self.ash = ash
      self.hue = hue
      self.proline = proline


In [22]:
Base.metadata.create_all(engine)
Base.query = db_session.query_property()
# session.query(Base)

2021-07-12 16:36:24,185 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-12 16:36:24,189 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("wine_class")
2021-07-12 16:36:24,192 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-12 16:36:24,200 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("wine_class")
2021-07-12 16:36:24,202 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-12 16:36:24,209 INFO sqlalchemy.engine.Engine 
CREATE TABLE wine_class (
	id INTEGER NOT NULL, 
	wine_class INTEGER, 
	alcohol FLOAT, 
	ash FLOAT, 
	hue FLOAT, 
	proline INTEGER, 
	PRIMARY KEY (id)
)


2021-07-12 16:36:24,211 INFO sqlalchemy.engine.Engine [no key 0.00192s] ()
2021-07-12 16:36:24,229 INFO sqlalchemy.engine.Engine COMMIT


In [23]:
def read_data():
   wine_df = pd.read_csv('wine_class.csv')

   for index, _df in wine_df.iterrows():
      row = Wine(wine_class=_df['Class'], alcohol=_df['Alcohol'], ash=_df['Ash'], hue=_df['Hue'], proline=_df['Proline'])
      db_session.add(row)
      # session.add(row)

   db_session.commit()
   # session.commit()

In [ ]:
read_data()

In [25]:
# CRUD
db = db_session.query(Wine).all()

2021-07-12 16:37:53,367 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-12 16:37:53,372 INFO sqlalchemy.engine.Engine SELECT wine_class.id AS wine_class_id, wine_class.wine_class AS wine_class_wine_class, wine_class.alcohol AS wine_class_alcohol, wine_class.ash AS wine_class_ash, wine_class.hue AS wine_class_hue, wine_class.proline AS wine_class_proline 
FROM wine_class
2021-07-12 16:37:53,374 INFO sqlalchemy.engine.Engine [generated in 0.00201s] ()


In [ ]:
for row in db:
   print(row.alcohol, row.ash)

In [30]:
# Basic
db = db_session.query(Wine.hue, Wine.proline).all()

2021-07-12 16:42:57,299 INFO sqlalchemy.engine.Engine SELECT wine_class.hue AS wine_class_hue, wine_class.proline AS wine_class_proline 
FROM wine_class
2021-07-12 16:42:57,302 INFO sqlalchemy.engine.Engine [cached since 154s ago] ()


In [31]:
# Filter
db = db_session.query(Wine).filter(Wine.hue > 1.0).all()
# session.query(Wine).filter_by(hue > 1.0)

2021-07-12 16:42:58,212 INFO sqlalchemy.engine.Engine SELECT wine_class.id AS wine_class_id, wine_class.wine_class AS wine_class_wine_class, wine_class.alcohol AS wine_class_alcohol, wine_class.ash AS wine_class_ash, wine_class.hue AS wine_class_hue, wine_class.proline AS wine_class_proline 
FROM wine_class 
WHERE wine_class.hue > ?
2021-07-12 16:42:58,216 INFO sqlalchemy.engine.Engine [cached since 20.99s ago] (1.0,)


In [32]:
# Limitation
db = db_session.query(Wine).limit(20).all()

2021-07-12 16:43:59,883 INFO sqlalchemy.engine.Engine SELECT wine_class.id AS wine_class_id, wine_class.wine_class AS wine_class_wine_class, wine_class.alcohol AS wine_class_alcohol, wine_class.ash AS wine_class_ash, wine_class.hue AS wine_class_hue, wine_class.proline AS wine_class_proline 
FROM wine_class
 LIMIT ? OFFSET ?
2021-07-12 16:43:59,888 INFO sqlalchemy.engine.Engine [generated in 0.00189s] (20, 0)


In [35]:
# Sort
db = db_session.query(Wine).order_by(desc(Wine.hue)).all()

2021-07-12 16:45:29,304 INFO sqlalchemy.engine.Engine SELECT wine_class.id AS wine_class_id, wine_class.wine_class AS wine_class_wine_class, wine_class.alcohol AS wine_class_alcohol, wine_class.ash AS wine_class_ash, wine_class.hue AS wine_class_hue, wine_class.proline AS wine_class_proline 
FROM wine_class ORDER BY wine_class.hue DESC
2021-07-12 16:45:29,310 INFO sqlalchemy.engine.Engine [cached since 2.314s ago] ()


In [36]:
# Create
wine = Wine(wine_class=1, alcohol=1, ash=1, hue=1, proline=1)
db_session.add(wine)
db_session.commit()

2021-07-12 16:50:48,848 INFO sqlalchemy.engine.Engine INSERT INTO wine_class (wine_class, alcohol, ash, hue, proline) VALUES (?, ?, ?, ?, ?)
2021-07-12 16:50:48,856 INFO sqlalchemy.engine.Engine [cached since 864s ago] (1, 1.0, 1.0, 1.0, 1)
2021-07-12 16:50:48,863 INFO sqlalchemy.engine.Engine COMMIT


In [37]:
# Update
db = db_session.query(Wine).filter(Wine.proline == 1).first()
db.wine_class = 10
db_session.commit()

2021-07-12 16:52:30,664 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-12 16:52:30,668 INFO sqlalchemy.engine.Engine SELECT wine_class.id AS wine_class_id, wine_class.wine_class AS wine_class_wine_class, wine_class.alcohol AS wine_class_alcohol, wine_class.ash AS wine_class_ash, wine_class.hue AS wine_class_hue, wine_class.proline AS wine_class_proline 
FROM wine_class 
WHERE wine_class.proline = ?
 LIMIT ? OFFSET ?
2021-07-12 16:52:30,671 INFO sqlalchemy.engine.Engine [generated in 0.00175s] (1, 1, 0)
2021-07-12 16:52:30,674 INFO sqlalchemy.engine.Engine UPDATE wine_class SET wine_class=? WHERE wine_class.id = ?
2021-07-12 16:52:30,677 INFO sqlalchemy.engine.Engine [generated in 0.00133s] (10, 179)
2021-07-12 16:52:30,681 INFO sqlalchemy.engine.Engine COMMIT


In [43]:
db_session.query(Wine).filter(Wine.proline == 1).first().wine_class

2021-07-12 16:53:51,438 INFO sqlalchemy.engine.Engine SELECT wine_class.id AS wine_class_id, wine_class.wine_class AS wine_class_wine_class, wine_class.alcohol AS wine_class_alcohol, wine_class.ash AS wine_class_ash, wine_class.hue AS wine_class_hue, wine_class.proline AS wine_class_proline 
FROM wine_class 
WHERE wine_class.proline = ?
 LIMIT ? OFFSET ?
2021-07-12 16:53:51,447 INFO sqlalchemy.engine.Engine [cached since 80.78s ago] (1, 1, 0)


10

In [44]:
# Delete
db_session.query(Wine).filter(Wine.proline == 1).delete()

2021-07-12 16:54:26,659 INFO sqlalchemy.engine.Engine DELETE FROM wine_class WHERE wine_class.proline = ?
2021-07-12 16:54:26,661 INFO sqlalchemy.engine.Engine [generated in 0.00236s] (1,)


1

In [46]:
for row in db_session.query(Wine).all():
   print(row.proline)

2021-07-12 16:56:11,946 INFO sqlalchemy.engine.Engine SELECT wine_class.id AS wine_class_id, wine_class.wine_class AS wine_class_wine_class, wine_class.alcohol AS wine_class_alcohol, wine_class.ash AS wine_class_ash, wine_class.hue AS wine_class_hue, wine_class.proline AS wine_class_proline 
FROM wine_class
2021-07-12 16:56:11,950 INFO sqlalchemy.engine.Engine [cached since 1099s ago] ()
1065
1050
1185
1480
735
1450
1290
1295
1045
1045
1510
1280
1320
1150
1547
1310
1280
1130
1680
845
780
770
1035
1015
845
830
1195
1285
915
1035
1285
1515
990
1235
1095
920
880
1105
1020
760
795
1035
1095
680
885
1080
1065
985
1060
1260
1150
1265
1190
1375
1060
1120
970
1270
1285
520
680
450
630
420
355
678
502
510
750
718
870
410
472
985
886
428
392
500
750
463
278
714
630
515
520
450
495
562
680
625
480
450
495
290
345
937
625
428
660
406
710
562
438
415
672
315
510
488
312
680
562
325
607
434
385
407
495
345
372
564
625
465
365
380
380
378
352
466
342
580
630
530
560
600
650
695
720
515
580
590
600
78